In [10]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import glob
from numba import jit, njit, prange
reload(ld)

<module 'sib_ldsc_z' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

In [11]:
np.random.seed(123)

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])

model = ld.sibreg(S = S)
model.simdata(V/N, N, simr = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [12]:
ld.neg_logll_grad(V, model.z, model.S, model.u, model.r)

C:\Users\Hariharan\Documents\git_repos\SNIPar\ldsc_reg\inferz\sib_ldsc_z.py:242: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, F), array(float64, 2d, A))
  log_ll[i] = (1/ui) * _log_ll(V_norm , zi, Si, ri)
C:\Users\Hariharan\Documents\git_repos\SNIPar\ldsc_reg\inferz\sib_ldsc_z.py:242: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, F), array(float64, 2d, A))
  log_ll[i] = (1/ui) * _log_ll(V_norm , zi, Si, ri)
C:\Users\Hariharan\Documents\git_repos\SNIPar\ldsc_reg\inferz\sib_ldsc_z.py:243: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  Gvec[i, :, :] = (1/ui) * _grad_ll_v(V_norm, zi, Si, ri)
C:\Users\Hariharan\Documents\git_repos\SNIPar\ldsc_reg\inferz\sib_ldsc_z.py:243: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  Gvec[i, :, :] = (1/ui) * _grad_ll_v(V_no

(38631.03860184946, array([[-140.90639708,  -82.11590935],
        [ -82.11590935,  125.08515464]]))

In [13]:
ld._log_ll(V, model.z[0, :], model.S[0], model.r[0])

C:\Users\Hariharan\Documents\git_repos\SNIPar\ldsc_reg\inferz\sib_ldsc_z.py:150: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, F), array(float64, 2d, A))
  - (1/2) * z.T @ Sigma_inv @ z)
C:\Users\Hariharan\Anaconda3\lib\site-packages\numba\typing\npydecl.py:967: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, F), array(float64, 2d, A))
  warnings.warn(NumbaPerformanceWarning(msg))


-11.68651716929529

In [47]:
model.solve()

No initial guess provided.
Making 'optimal' matrix


(array([[0.52262128, 0.26207389],
        [0.26207389, 0.48484274]]),       fun: 38627.52359063335
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 2.43951351e-05, -1.33080442e-05,  7.36192092e-05])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 48
       nit: 17
    status: 0
   success: True
         x: array([0.52262128, 0.26207389, 0.48484274]))

In [ ]:
def extract_bounds(n):
    
    # =============================== #
    # From a number n, the function
    # outputs a list of bounds
    # for a var cov matrix of size
    # n x n
    # =============================== #
    
    # extract idx of flat array whcih are diagonals
    uptriangl_idx = np.array(np.triu_indices(n))
    diags = uptriangl_idx[0, :] == uptriangl_idx[1, :]
    
    # Construct list of bounds
    bounds_list = np.array([(None, None)] * len(diags))
    bounds_list[diags] = (1e-6, None)
    
    bounds_list_out = [tuple(i) for i in bounds_list]
    
    return bounds_list_out


In [ ]:
@njit
def calc_inv_root(S):
    '''
    A stable solver for S^{-1/2}
    '''
    
    if ~np.any(np.isnan(S)):
        S_eig = np.linalg.eig(S)
        l = np.zeros(S.shape)
        np.fill_diagonal(l,np.power(S_eig[0],-0.5))
        S_inv_root = S_eig[1].dot(np.dot(l,S_eig[1].T))
    else:
        S_inv_root =  np.empty_like(S)
        S_inv_root[:] = np.nan
    return S_inv_root

In [ ]:
def extract_upper_triangle(x):
    
    # =============================== #
    # Extracts the upper triangular portion of 
    # a symmetric matrix
    # =============================== #
    
    n, m = x.shape
    assert n == m
    
    upper_triangle = x[np.triu_indices(n)]
    
    return upper_triangle

def return_to_symmetric(triangle_vec, final_size):
    
    # =============================== #
    # Given a vector of the upper triangular matrix,
    # get back the symmetric matrix
    # =============================== #
    
    X = np.zeros((final_size,final_size))
    X[np.triu_indices(X.shape[0], k = 0)] = triangle_vec
    X = X + X.T - np.diag(np.diag(X))
    
    return X

In [3]:
@njit
def _log_ll_vec(V, z, S, r):

    """
    Returns the log likelihood matrix for a given SNP i as formulated by:

    .. math::
        l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) -
                \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i

    Inputs:
    V = dxd numpy matrix
    z = dx1 numpy matrix
    S = dxd numpy matrix
    r = scalar
    f = scalar

    Outputs:
    logll = scalar
    """

    S_inv_root = calc_inv_root(S)
    Sigma = np.identity(S.shape[0])+r*np.dot(S_inv_root.dot(V),S_inv_root)
    logdet = np.linalg.slogdet(Sigma)

    det = np.linalg.det(Sigma)
    if det > 1e-6 or det < -1e-6:
        Sigma_inv = np.linalg.inv(Sigma)
    else:
        Sigma_inv = np.linalg.pinv(Sigma)

    z = z.reshape(V.shape[0],1)
    d = V.shape[0]

    L = - (d/2) * np.log(2 * np.pi) \
        - (1/2) * logdet[0]*logdet[1] \
        - (1/2) * np.dot(z.T,Sigma_inv.dot(z))

    return L[0, 0]

NameError: name 'njit' is not defined

In [4]:
@njit
def _grad_ll_v(V, z, S, r):

    """
    Returns the gradient of the log likelihood wrt V for a given SNP i as formulated by:

    .. math::
        \frac{dl}{dV} = S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}

    Inputs:
    V = dxd numpy matrix
    z = dx1 numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix

    Outputs:
    grad_ll_v = dxd matrix 
    """

    S_inv_root = calc_inv_root(S)
    Sigma = np.identity(S.shape[0])+r*np.dot(S_inv_root.dot(V),S_inv_root)

    det = np.linalg.det(Sigma)
    if det > 1e-6 or det < -1e-6:
        Sigma_inv = np.linalg.inv(Sigma)
    else:
        Sigma_inv = np.linalg.pinv(Sigma)

    z = z.reshape(z.shape[0],1)
    SSigma_inv = S_inv_root.dot(Sigma_inv)
    g = r * SSigma_inv.dot(np.dot(Sigma-z.dot(z.T),SSigma_inv.T))
    return -(1/2) * g

NameError: name 'njit' is not defined

In [8]:
@njit(parallel = True)
def neg_logll_grad(V, 
                   z, S, 
                   u, r, 
                   f):

    """
    Returns the loglikelihood and its gradient wrt V for a given SNP i as formulated by:

    .. math::
        l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) -
                \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i

    and

    .. math::
        \frac{dl}{dV} = S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}

    Inputs:
    V = dxd numpy matrix
    z = dxN numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix
    logllfunc = function which calculates logll
                (uses self._log_ll by default)
    gradfunc = function which calculated grad of logll
                (uses self._grad_ll_v by default)

    Outputs:
    -log_ll = 1x1 scalar
    -Gvec = dxd numpy matrix
    """

    # Unflatten V into a matrix
    d = S[0].shape[0]
    N = len(S)
    
    Gvec = np.zeros((N, d, d))
    log_ll = np.zeros(N)

    # Normalizg variables
    V_norm = V
    for i in prange(N):

        Si = S[i]
        zi = z[i, :].reshape((d, 1))
        ui = u[i]
        ri = r[i]

        Si = N * Si

        log_ll[i] = (1/ui) * _log_ll_vec(V_norm , zi, Si, ri)
        Gvec[i, :, :] = (1/ui) * _grad_ll_v(V_norm, zi, Si, ri)

    return -log_ll.sum() , -Gvec.sum(axis = 0)

In [9]:
def neg_logll_grad_basic(V, 
                           z, S, 
                           u, r, 
                           f):

    """
    Returns the loglikelihood and its gradient wrt V for a given SNP i as formulated by:

    .. math::
        l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) -
                \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i

    and

    .. math::
        \frac{dl}{dV} = S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}

    Inputs:
    V = dxd numpy matrix
    z = dxN numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix
    logllfunc = function which calculates logll
                (uses self._log_ll by default)
    gradfunc = function which calculated grad of logll
                (uses self._grad_ll_v by default)

    Outputs:
    -log_ll = 1x1 scalar
    -Gvec = dxd numpy matrix
    """

    # Unflatten V into a matrix
    d = S[0].shape[0]
    N = len(S)
    
    Gvec = np.zeros((N, d, d))
    log_ll = np.zeros(N)

    # Normalizg variables
    V_norm = V
    for i in range(N):

        Si = S[i]
        zi = z[i, :].reshape((d, 1))
        ui = u[i]
        ri = r[i]

        Si = N * Si

        log_ll[i] = (1/ui) * _log_ll_vec(V_norm , zi, Si, ri)
        Gvec[i, :, :] = (1/ui) * _grad_ll_v(V_norm, zi, Si, ri)

    return log_ll.sum() , Gvec.sum(axis = 0)

In [19]:
def neglike_wrapper(V, z, S, u, r, f):
    
    d = S[0].shape[0]
    
    V = return_to_symmetric(V, d)
    
    logll, Gvec = neg_logll_grad(V, 
                               z, S, 
                               u, r, 
                               f)
    
    Gvec = extract_upper_triangle(Gvec)
    
    return logll, Gvec
    

In [1]:
S

NameError: name 'S' is not defined

In [11]:
_log_ll_vec(V, model.z[0], model.S[0], model.r[0])

<ipython-input-6-5712ba8a02bc>:37: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, C))
  - (1/2) * np.dot(z.T,Sigma_inv.dot(z))


-11.68651716929529

In [13]:
_grad_ll_v(V, model.z[0], model.S[0], model.r[0])

<ipython-input-7-a14bcf696e6f>:32: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  SSigma_inv = S_inv_root.dot(Sigma_inv)


array([[-1.33298951,  0.66649537],
       [ 0.66649537, -1.33316706]])

In [14]:
logll, gvec = neg_logll_grad(V, model.z, model.S, model.u, model.r, model.f)
print(logll, gvec)

38631.03860184946 [[-140.90639708  -82.11590935]
 [ -82.11590935  125.08515464]]


In [15]:
neg_logll_grad_basic(V, model.z, model.S, model.u, model.r, model.f)

(-38631.03860184951, array([[ 140.90639708,   82.11590935],
        [  82.11590935, -125.08515464]]))

In [16]:
neglike_wrapper(extract_upper_triangle(V), model.z, model.S, model.u, model.r, model.f)

(-38631.03860184946, array([ 140.90639708,   82.11590935, -125.08515464]))

In [20]:
def solve(z, 
          S,
          u,
          r,
          f,
         est_init = None,
         printout = True):

    """
    Solves the ldsc problem of infering the V matrix

    Inputs:
    z = Nx1 numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix

    Outputs:
    output_matrix = dxd numpy matrix
    result = result of scipy solver 
    """
    # == Solves our MLE problem == #
    n, m = z.shape

    if est_init is not None:
        # Shape of initial varcov guess
        rowstrue = est_init.shape[0] == m
        colstrue = est_init.shape[1] == m

        if rowstrue & colstrue:
            pass
        else:
            if printout == True:
                print("Warning: Initial Estimate given is not of the proper dimension")
                print("Making 'optimal' matrix")
                print("=================================================")

            est_init = np.zeros((m, m))
    else:
        if printout == True:
            print("No initial guess provided.")
            print("Making 'optimal' matrix")
            print("=================================================")

        est_init = np.zeros((m, m))


    # exporting for potential later reference
#     self.est_init = est_init

    # extract array from est init
    est_init_array = extract_upper_triangle(est_init) 

    bounds = extract_bounds(m)     

    result = minimize(
        neglike_wrapper, 
        est_init_array,
        jac = True,
        args = (z, S, u, r, f),
        bounds = bounds,
        method = 'L-BFGS-B',
        options = {'ftol' : 1e-15}
    )

    output_matrix = return_to_symmetric(result.x, m)

    # re-normnalizing output matrix 
#     self.output_matrix = output_matrix

    return output_matrix, result 

In [21]:
solve(model.z, model.S, model.u, model.r, model.f)

No initial guess provided.
Making 'optimal' matrix


(array([[0.52262128, 0.26207389],
        [0.26207389, 0.48484274]]),       fun: 38627.52359063335
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 2.43951351e-05, -1.33080442e-05,  7.36192092e-05])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 48
       nit: 17
    status: 0
   success: True
         x: array([0.52262128, 0.26207389, 0.48484274]))